In [1]:
import numpy as np
import pandas as pd
import sklearn
import torch

from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F
import torch.optim as optim

In [2]:
batch_size=64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])
train_dataset=datasets.MNIST('./data',train=True,download=True,transform=transform)
test_dataset=datasets.MNIST('./data',train=False,download=True,transform=transform)

train_loader=DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
test_loader=DataLoader(test_dataset,shuffle=False,batch_size=batch_size)

In [3]:
class CNN_net(torch.nn.Module):
    def __init__(self):
        super(CNN_net,self).__init__()#(batch,1,28,28)
        self.conv1=torch.nn.Conv2d(1,10,kernel_size=5)#(batch,10，24，24)
        self.pooling=torch.nn.MaxPool2d(2) #(batch,10,12,12)
        self.conv2=torch.nn.Conv2d(10,20,kernel_size=5)#(batch,20,8,8)
        self.pooling=torch.nn.MaxPool2d(2) #(batch,20,4,4)
        self.fc=torch.nn.Linear(320,10) #(batch,320)
            
    def forward(self,x):
        batch_size=x.size(0) #batch_size = 64
#         print("batch_size",batch_size)
        x=F.relu(self.pooling(self.conv1(x)))#(batch,10,12,12)
        x=F.relu(self.pooling(self.conv2(x)))#(batch,20,4,4)
        x=x.view(batch_size,-1)#(batch,320)
#         print("x",x.size())

        x=self.fc(x)
        return x

In [4]:
model = CNN_net()
criterion=torch.nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.01,momentum=0.5) #momentum??

# device = torch.device("cuda:0" if torch.cuda.is_available()else"cpu")
# model.to(device)

In [5]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate (train_loader, 0):
        inputs, target = data
#        inputs, target = inputs.to(device), target.to(device) #gpu
        optimizer.zero_grad()
        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
                                      
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print ('[%d, %5d] loss: %.3f' % (epoch + 1 , batch_idx + 1 , running_loss / 300))
            running_loss = 0.0

In [6]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim= 1)
            total += labels.size( 0)
            correct += (predicted == labels).sum().item()
    print ('Accuracy on test set: %d %%' % (100 * correct /total))

In [7]:
for epoch in range (3):
    train(epoch)
    test()

[1,   300] loss: 0.596
[1,   600] loss: 0.208
[1,   900] loss: 0.144
Accuracy on test set: 96 %
[2,   300] loss: 0.112
[2,   600] loss: 0.099
[2,   900] loss: 0.093
Accuracy on test set: 97 %
[3,   300] loss: 0.084
[3,   600] loss: 0.070
[3,   900] loss: 0.070
Accuracy on test set: 98 %
